## LLM

In [1]:
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output
from src.tokenizer import TokenizerBPE, TokenizerChar, word_split, normalize_to_ascii, pair_freq
from src.data_handling import read_first_n, normalize_to_ascii


import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import numpy as np
import pickle as pkl
from tqdm.notebook import tqdm
from src.data_handling import fused_qa
import sys
import random
import json

# disable gpu for testing purposes
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [ ]:
"""
with open('corpus/triviaqa-rc/qa/web-train.json', 'r', encoding='utf-8') as f:
    squad = json.load(f)

question_web_list = []
answer_web_list = []
evidence_web_list = []

for qa in tqdm(squad['Data']):
    question_web_list.append(qa['Question'])
    answer_web_list.append(qa['Answer']["Value"])
    sr = qa['SearchResults']
    if len(sr) > 0: 
        evidence_web_list.append(sr[0]["Filename"])
    else:
        evidence_web_list.append(None)
"""

  0%|          | 0/76496 [00:00<?, ?it/s]

In [ ]:
"""

path = "corpus/triviaqa-rc/evidence/web/"

txt_list = []

for evidence in tqdm(evidence_web_list):
    if evidence is not None:
        txt = open(path + evidence, 'r', encoding='utf-8').read()
        txt_list.append(txt)
"""   

  0%|          | 0/76496 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
txt = "".join(txt_list)
print("Length of txt: ", len(txt))

Length of txt:  1163223076


In [3]:
with open('corpus/triviaqa-rc/qa/wikipedia-train.json', 'r', encoding='utf-8') as f:
    squad = json.load(f)

question_wiki_list = []
answer_wiki_list = []
evidence_wiki_list = []

for qa in tqdm(squad['Data']):
    question_wiki_list.append(qa['Question'])
    answer_wiki_list.append(qa['Answer']["Value"])
    sr = qa["EntityPages"]
    if len(sr) > 0: 
        evidence_wiki_list.append(sr[0]["Filename"])
    else:
        evidence_wiki_list.append(None)
    

  0%|          | 0/61888 [00:00<?, ?it/s]

In [4]:
path = "corpus/triviaqa-rc/evidence/wikipedia/"

evidence_article = {}
seen = {}

for i, evidence in tqdm(list(enumerate(evidence_wiki_list))):
    if evidence is not None:
        evidence = evidence.replace(":", "_").replace("?", "_").replace("*", "_").replace('"', "_")
        if not evidence in seen:
            seen[evidence] = 1
            try:
                txt = open(path + evidence, 'r', encoding='utf-8').read()
                evidence_article[evidence] = txt
            except:
                print("Error reading file: ", evidence)
                continue
        else:
            seen[evidence] += 1

  0%|          | 0/61888 [00:00<?, ?it/s]

In [6]:
rank = [[evidence, freq] for freq, evidence in sorted(zip(seen.values(), seen.keys()), reverse=True)]
print(len(rank))
print(rank[1990:2000])

28406
[['Dwarf_planet.txt', 5], ['Dusty_Springfield.txt', 5], ['Dunkirk_evacuation.txt', 5], ['Duke_of_Northumberland.txt', 5], ['Dubris.txt', 5], ['Draughts.txt', 5], ['Donna_Summer.txt', 5], ['Don_McLean.txt', 5], ['Dodoma.txt', 5], ['Dodo.txt', 5]]


In [7]:
def get_top_n_evidence(n, rank):
    top_n = []
    for i in range(n):
        article = evidence_article[rank[i][0]]
        top_n.append(article)
    return top_n

In [8]:
article_wiki_list = get_top_n_evidence(2000, rank)

text = "".join(article_wiki_list)
print("article_list: ", len(text))

article_list:  80246283


In [9]:
text = "".join(question_wiki_list + answer_wiki_list)
print("Length of question_web_list: ", len(text))

Length of question_web_list:  5580683


In [ ]:
random.seed(42)

question_wiki_clean_list = [normalize_to_ascii(x) for x in question_wiki_list]
answer_wiki_clean_list = [normalize_to_ascii(x) for x in answer_wiki_list]
article_wiki_clean_list = [normalize_to_ascii(x) for x in article_wiki_list]

corpus = question_wiki_clean_list + answer_wiki_clean_list + article_wiki_clean_list

#tokenizer = TokenizerChar(corpus)

tokenizer = TokenizerBPE(corpus, 16000, lowercase=True)
tokenizer.destroy_hash()

pkl.dump(tokenizer, open('tokenizers/trivia_16k.pkl', 'wb'))

Lowercasing corpus


  0%|          | 0/125776 [00:00<?, ?it/s]

Merging tokens


  0%|          | 0/16000 [00:00<?, ?it/s]

t h 1739722
i n 1368586
th e 1260354
a n 1155479
e r 1026645
o n 927284
r e 789480
a t 688305
e n 665902
e d 658304
o r 628376
a l 626054
s t 624749
a r 589821
o f 544347
an d 523224
i s 485334
a s 476460
e s 466066
i t 439931
i c 435224
in g 394414
t o 384346
r o 364890
i on 338153
l e 313280
o u 311515
c h 268346
en t 244211
i l 232653
a m 229265
s e 225895
w h 206724
e l 206621
c o 200206
a d 197802
a c 189376
b e 182225
f or 176864
l y 175310
d e 165925
u r 159442
u n 157746
o l 154048
i m 148475
i v 145990
i r 144233
w as 142257
at ion 137689
i g 136940
u s 131592
t er 130518
c on 130306
o w 130204
c e 127849
w i 121107
v er 117437
b y 117226
1 9 117199
i d 116013
a y 115688
m o 113961
a g 113063
s u 111344
c t 109803
m e 109118
p e 108672
' s 107845
h e 107397
wi th 106561
a re 99559
u l 99532
t e 99071
i st 98456
t r 98356
s o 97701
a p 96443
er s 93748
n e 93501
p o 92775
ic h 91088
a v 91072
a b 90899
th at 89343
co m 89253
al l 88695
p l 88212
p ro 87008
re s 86725
m an 85939

In [47]:
tokenizer.token_to_idx

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '#': 5,
 '$': 6,
 '%': 7,
 '&': 8,
 "'": 9,
 '(': 10,
 ')': 11,
 '*': 12,
 '+': 13,
 ',': 14,
 '-': 15,
 '.': 16,
 '/': 17,
 '0': 18,
 '1': 19,
 '2': 20,
 '3': 21,
 '4': 22,
 '5': 23,
 '6': 24,
 '7': 25,
 '8': 26,
 '9': 27,
 ':': 28,
 ';': 29,
 '<': 30,
 '=': 31,
 '>': 32,
 '?': 33,
 '@': 34,
 'A': 35,
 'B': 36,
 'C': 37,
 'D': 38,
 'E': 39,
 'F': 40,
 'G': 41,
 'H': 42,
 'I': 43,
 'J': 44,
 'K': 45,
 'L': 46,
 'M': 47,
 'N': 48,
 'O': 49,
 'P': 50,
 'Q': 51,
 'R': 52,
 'S': 53,
 'T': 54,
 'U': 55,
 'V': 56,
 'W': 57,
 'X': 58,
 'Y': 59,
 'Z': 60,
 '[': 61,
 '\\': 62,
 ']': 63,
 '^': 64,
 '_': 65,
 '`': 66,
 'a': 67,
 'b': 68,
 'c': 69,
 'd': 70,
 'e': 71,
 'f': 72,
 'g': 73,
 'h': 74,
 'i': 75,
 'j': 76,
 'k': 77,
 'l': 78,
 'm': 79,
 'n': 80,
 'o': 81,
 'p': 82,
 'q': 83,
 'r': 84,
 's': 85,
 't': 86,
 'u': 87,
 'v': 88,
 'w': 89,
 'x': 90,
 'y': 91,
 'z': 92,
 '{': 93,
 '|': 94,
 '}': 95,
 '~': 96,
 '\x80': 97,
 '\x85': 98,
 '\x91': 99

In [ ]:
def fused_qa(question_list, answer_list):
    q ="<q>"
    a = "<a>"
    sos = "<s>"
    eos = "</s>"

    corpus_list = []
    for question, answer in tqdm(list(zip(question_list, answer_list))):
        qa = [sos, q, question, a, answer, eos]
        corpus_list.extend(qa)
    
    return "".join(corpus_list)


def fused_article(corpus_list):
    sos = "<s>"
    eos = "</s>"

    corpus_padded_list = []
    for line in corpus_list:
        line = [sos, line, eos]
        corpus_padded_list.extend(line)
    
    return "".join(corpus_padded_list)

In [10]:
tokenizer = pkl.load(open('tokenizers/CNN_QA8k_balanced.pkl', 'rb'))
tokenizer.create_hash()

tokenizer.add_special_tokens(["<s>", "</s>", "<q>", "<a>",])

In [ ]:
corpus_qa_fused = fused_qa(question_wiki_clean_list, answer_wiki_clean_list)

corpus_qa_encoded = tokenizer.encode(corpus_qa_fused)

pkl.dump(corpus_qa_encoded, open('corpus/trivia_qa_16k_encoded', 'wb'))

  0%|          | 0/86820 [00:00<?, ?it/s]

  0%|          | 0/8000 [00:00<?, ?it/s]

In [ ]:
corpus_article_fused = fused_article(article_wiki_clean_list)

corpus_article_encoded = tokenizer.encode(corpus_article_fused)

pkl.dump(corpus_article_encoded, open('corpus/trivia_article_16k_encoded', 'wb'))

  0%|          | 0/8000 [00:00<?, ?it/s]

## 